In [1]:
import pandas as pd
import numpy as np
import json
import csv
import requests
import datetime as dt
from sqlalchemy import create_engine
from sqlalchemy import MetaData

In [122]:
def connect_db(database='fees'):
    user='root'
    password='240699'
    host='localhost'
    database=database
    

    # Connect to the database
    engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")
    try:
        connection = engine.connect()
        return connection
    except Exception as e:
        raise e

In [123]:
response = requests.get('https://api.llama.fi/overview/fees?excludeTotalDataChart=false&excludeTotalDataChartBreakdown=false&dataType=dailyFees')
data = response.json()

In [86]:
data_chart = data['totalDataChart']
breakdown = data['totalDataChartBreakdown']


In [87]:
fees_historic = pd.DataFrame(data_chart, columns=['date', 'fees'])
fees_historic['date'] = pd.to_datetime(fees_historic['date'], unit='s')

In [88]:
# Calculate percentage changes for different time intervals
fees_historic['1d_change'] = fees_historic['fees'].pct_change() * 100
fees_historic['7d_change'] = fees_historic['fees'].pct_change(periods=7) * 100
fees_historic['30d_change'] = fees_historic['fees'].pct_change(periods=30) * 100
fees_historic['90d_change'] = fees_historic['fees'].pct_change(periods=90) * 100
fees_historic['180d_change'] = fees_historic['fees'].pct_change(periods=180) * 100
fees_historic['1y_change'] = fees_historic['fees'].pct_change(periods=365) * 100

#### Rolling Statistical

In [89]:
fees_historic['rolling_mean_10'] = fees_historic['fees'].rolling(window=10).mean()
fees_historic['rolling_mean_25'] = fees_historic['fees'].rolling(window=25).mean()
fees_historic['rolling_mean_50'] = fees_historic['fees'].rolling(window=50).mean()
fees_historic['rolling_mean_100'] = fees_historic['fees'].rolling(window=100).mean()


In [90]:
fees_historic['rolling_std_10'] = fees_historic['fees'].rolling(window=10).std()
fees_historic['rolling_std_25'] = fees_historic['fees'].rolling(window=25).std()
fees_historic['rolling_std_50'] = fees_historic['fees'].rolling(window=50).std()
fees_historic['rolling_std_100'] = fees_historic['fees'].rolling(window=100).std()


In [91]:
fees_historic['rolling_min_10'] = fees_historic['fees'].rolling(window=10).min()
fees_historic['rolling_min_25'] = fees_historic['fees'].rolling(window=25).min()
fees_historic['rolling_min_50'] = fees_historic['fees'].rolling(window=50).min()
fees_historic['rolling_min_100'] = fees_historic['fees'].rolling(window=100).min()

In [92]:
fees_historic['rolling_max_10'] = fees_historic['fees'].rolling(window=10).max()
fees_historic['rolling_max_25'] = fees_historic['fees'].rolling(window=25).max()
fees_historic['rolling_max_50'] = fees_historic['fees'].rolling(window=50).max()
fees_historic['rolling_max_100'] = fees_historic['fees'].rolling(window=100).max()

#### Volatility

In [93]:
fees_historic['historical_volatility'] = fees_historic['fees'].pct_change().rolling(window=10).std() * np.sqrt(252)


In [94]:
fees_historic['ema_12'] = fees_historic['fees'].ewm(span=12, adjust=False).mean()
fees_historic['ema_26'] = fees_historic['fees'].ewm(span=26, adjust=False).mean()
fees_historic['macd'] = fees_historic['ema_12'] - fees_historic['ema_26']
fees_historic['signal_line'] = fees_historic['macd'].ewm(span=9, adjust=False).mean()

In [95]:
delta = fees_historic['fees'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
fees_historic['rsi'] = 100 - (100 / (1 + rs))

In [96]:
fees_historic['rate_of_change'] = fees_historic['fees'].pct_change() * 100
fees_historic['momentum'] = fees_historic['fees'].diff()

In [97]:
fees_historic['sma_20'] = fees_historic['fees'].rolling(window=20).mean()
fees_historic['upper_band'] = fees_historic['sma_20'] + 2 * fees_historic['fees'].rolling(window=20).std()
fees_historic['lower_band'] = fees_historic['sma_20'] - 2 * fees_historic['fees'].rolling(window=20).std()


In [98]:
fees_historic['stoch_oscillator'] = 100 * ((fees_historic['fees'] - fees_historic['fees'].rolling(window=14).min()) / 
                                           (fees_historic['fees'].rolling(window=14).max() - fees_historic['fees'].rolling(window=14).min()))


In [104]:
with connect_db() as connection:
    fees_historic.to_sql( 'historic', con = connection, if_exists = 'replace' )
    

In [105]:
df = pd.DataFrame(breakdown, columns=['date', 'id'])
df['date'] = pd.to_datetime(df['date'], unit='s')
df = df.set_index('date')

In [106]:
df = pd.DataFrame(df['id'].tolist(), index=df.index)

In [107]:
dfs = []
count = 0
with connect_db() as connection:
    for column in df.columns:
        # Create an individual DataFrame for each column
        individual_df = pd.DataFrame({
            'date': df.index,
            'id': column,
            'fees': df[column],
            '1d_change': df[column].pct_change() * 100,
            '7d_change': df[column].pct_change(periods=7) * 100,
            '30d_change': df[column].pct_change(periods=30) * 100,
            '90d_change': df[column].pct_change(periods=90) * 100,
            '180d_change': df[column].pct_change(periods=180) * 100,
            '1y_change': df[column].pct_change(periods=365) * 100,

            'ema_10': df[column].ewm(span=10, adjust=False).mean(),
            'ema_25': df[column].ewm(span=25, adjust=False).mean(),
            'ema_50': df[column].ewm(span=50, adjust=False).mean(),
            'ema_100': df[column].ewm(span=100, adjust=False).mean(),

            'rolling_std_10': df[column].rolling(window=10).std(),
            'rolling_std_25': df[column].rolling(window=25).std(),
            'rolling_std_50': df[column].rolling(window=50).std(),
            'rolling_std_100': df[column].rolling(window=100).std(),
            
            'rolling_min_10': df[column].rolling(window=10).min(),
            'rolling_max_10': df[column].rolling(window=10).max(),
            'rolling_min_25': df[column].rolling(window=25).min(),
            'rolling_max_25': df[column].rolling(window=25).max(),
            'rolling_min_50': df[column].rolling(window=50).min(),
            'rolling_max_50': df[column].rolling(window=50).max(),
            'rolling_min_100': df[column].rolling(window=100).min(),
            'rolling_max_100': df[column].rolling(window=100).max(),

            'historical_volatility': df[column].pct_change().rolling(window=10).std() * np.sqrt(252),
            
            'ema_12': df[column].ewm(span=12, adjust=False).mean(),
            'ema_26': df[column].ewm(span=26, adjust=False).mean(),
            'macd': df[column].ewm(span=12, adjust=False).mean() - df[column].ewm(span=26, adjust=False).mean(),
            'signal_line': (df[column].ewm(span=12, adjust=False).mean() - df[column].ewm(span=26, adjust=False).mean()).ewm(span=9, adjust=False).mean(),

            'rsi': 100 - (100 / (1 + df[column].diff().where(df[column].diff() > 0, 0).rolling(window=14).mean() / df[column].diff().where(df[column].diff() < 0, 0).rolling(window=14).mean())),

            'momentum': df[column].diff(),
            
            'sma_20': df[column].rolling(window=20).mean(),
            'upper_band': df[column].rolling(window=20).mean() + 2 * df[column].rolling(window=20).std(),
            'lower_band': df[column].rolling(window=20).mean() - 2 * df[column].rolling(window=20).std(),


            'log_return': np.log(df[column] / df[column].shift(1)) * 100,
        })
        individual_df.replace([np.inf, -np.inf], np.nan, inplace=True)
        name = str(individual_df['id'].iloc[0])
        table_name = str(name).replace(' ', '_').lower()
        count+=1
        individual_df.to_sql( table_name + '_historic', con = connection, if_exists = 'replace', index = False )
        print(name + ' inserted.')
        # Append the individual DataFrame to the list
        dfs.append(individual_df)


Bitcoin inserted.
Compound inserted.
Doge inserted.
Ethereum inserted.
Litecoin inserted.
Opensea V1 inserted.


c:\Users\34633\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:405: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Uniswap V1 inserted.
MakerDAO inserted.
Canto inserted.
Cronos inserted.
Gnosis inserted.
Curve DEX inserted.
Tron inserted.
GhostMarket inserted.
Balancer V1 inserted.
Uniswap V2 inserted.
Celo inserted.
Polygon inserted.
Aurora inserted.
Near inserted.
Fantom inserted.
BSC inserted.
Honeyswap inserted.
SushiSwap inserted.
Quickswap V2 inserted.
AAVE V2 inserted.
Lido inserted.
MooniSwap inserted.
Solana inserted.
Foundation inserted.
Pangolin inserted.
ApeSwap AMM inserted.
Elk inserted.
KyberSwap - Classic inserted.
Liquity inserted.
Thorchain inserted.
SpookySwap inserted.
Balancer V2 inserted.
PancakeSwap AMM inserted.
PancakeSwap StableSwap inserted.
Uniswap V3 inserted.
SpiritSwap AMM inserted.
Convex Finance inserted.
Waves inserted.
Mimo inserted.
ShibaSwap inserted.
Hydradex V2 inserted.
Trader Joe DEX inserted.
Joe V2 inserted.
Synapse inserted.
Moonriver inserted.
GMX V1 inserted.
Abracadabra inserted.
Solarbeam inserted.
Defi Swap inserted.
BiSwap V2 inserted.
DefiPlaza in

c:\Users\34633\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:405: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Azuro inserted.
Pika inserted.
BetSwirl inserted.
MUX Protocol inserted.
MM Stableswap Polygon inserted.
Arbitrum inserted.
Gains Network inserted.
BabyDogeSwap inserted.
Stargate inserted.
Ferro inserted.
Quickswap V3 inserted.
Tigris inserted.
Frax Ether inserted.
VVS Standard inserted.
Gearbox inserted.
Osmosis DEX inserted.
Raydium inserted.
Tarot inserted.
Lifinity V1 inserted.
BlueMove Staking inserted.
Hegic inserted.
Y2K V1 inserted.
Llamalend inserted.
PepeTeam sWAVES inserted.
Avalanche inserted.
Concentrator inserted.
Camelot V2 inserted.
HMX inserted.
Archly V1 inserted.
Alchemix inserted.
Aura inserted.
ZORA inserted.
DODO inserted.
KyotoSwap inserted.
KPerp Exchange inserted.
Equalizer Exchange inserted.
X2Y2 inserted.
Mummy Finance inserted.
Level Finance inserted.
Vesta Finance inserted.
Beethoven X inserted.
Plenty inserted.
Solidly V2 inserted.
MetaMask inserted.
Perpetual Protocol inserted.
Rainbow inserted.
Sudoswap V1 inserted.
Blur Bids inserted.
Thena V1 inserted

In [79]:
df


,Bitcoin,Compound,Doge,Ethereum,Litecoin,Opensea V1,Uniswap V1,MakerDAO,Canto,Cronos,...,JustLend,Storm Trade,Metavault Derivatives V2,Butter.xyz,Pharaoh Exchange,Bluefin,Dexter,SquaDeFi,Surf Protocol,Shoebill V2
date,,,,,,,,,,,,,,,,,,,,,
2019-10-11,2.897206e+05,14413.043083,69.599390,9.556445e+04,575.553278,427.336151,2788.680506,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-12,2.304057e+05,14183.369765,69.805285,8.500889e+04,603.851885,372.967224,1108.347552,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-13,1.686948e+05,14280.334048,69.449317,7.059603e+04,494.780679,5551.176860,1276.540076,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-14,1.366041e+05,14191.084244,85.158109,6.830201e+04,475.498538,7146.317482,2358.515242,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-10-15,2.013097e+05,14391.165232,81.277121,8.294494e+04,544.798659,6778.663886,1994.336388,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-29,9.754698e+06,116095.060220,11548.437571,8.851393e+06,1616.680843,NaN,27.455886,487644.461162,NaN,10999.238255,...,11374.506472,1160.89,36.083141,3407.807582,7243.309317,24749.132644,87.492106,0.851565,174.316755,NaN
2023-12-30,8.941390e+06,100529.539997,4725.176131,6.989229e+06,1424.134569,NaN,20.771043,490323.835182,NaN,7778.080154,...,11451.624920,2127.19,0.013498,4117.829772,3777.380249,25138.409096,69.970200,0.000000,67.219485,NaN
2023-12-31,1.109526e+07,93262.206814,7261.582492,5.472280e+06,1753.531341,NaN,5.642742,491628.663941,NaN,7040.138627,...,13110.664942,596.30,1.489496,2861.666629,6607.857188,76498.460578,83.343655,0.000000,390.366930,NaN


In [60]:
data.keys()

dict_keys(['totalDataChart', 'totalDataChartBreakdown', 'protocols', 'allChains', 'chain', 'total24h', 'total48hto24h', 'total7d', 'total14dto7d', 'total60dto30d', 'total30d', 'total1y', 'average1y', 'change_1d', 'change_7d', 'change_1m', 'totalVolume7d', 'totalVolume30d', 'change_7dover7d', 'change_30dover30d', 'breakdown24h', 'dailyRevenue', 'dailyUserFees', 'dailyHoldersRevenue', 'dailySupplySideRevenue', 'dailyProtocolRevenue', 'dailyBribesRevenue', 'dailyTokenTaxes'])

In [108]:
df = pd.DataFrame(data['protocols'])

In [114]:
df.module

0      ArbitrumExchange
1      ArbitrumExchange
2               DerpDEX
3                 Scale
4               SmarDex
             ...       
339             smbswap
340             smbswap
341             uniswap
342             uniswap
343             uniswap
Name: module, Length: 344, dtype: object

In [118]:
df.defillamaId = df.defillamaId.astype(int)
df.name = df.name.astype(str)
df.displayName = df.displayName.astype(str)
df.module = df.module.astype(str)
df.logo = df.logo.astype(str)


In [119]:
df.dtypes

defillamaId                 int32
name                       object
disabled                     bool
displayName                object
module                     object
category                   object
logo                       object
change_1d                 float64
change_7d                 float64
change_1m                 float64
change_7dover7d           float64
change_30dover30d         float64
total24h                  float64
total48hto24h             float64
total7d                   float64
total30d                  float64
total14dto7d              float64
total60dto30d             float64
total1y                   float64
average1y                 float64
totalAllTime              float64
breakdown24h               object
chains                     object
protocolType               object
methodologyURL             object
methodology                object
parentProtocol             object
latestFetchIsOk              bool
versionKey                 object
dailyRevenue  

In [124]:
response = requests.get('https://api.llama.fi/overview/fees?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyFees')
data = response.json()

In [126]:
data = data['protocols']

In [129]:
data[0].keys()

dict_keys(['defillamaId', 'name', 'disabled', 'displayName', 'module', 'category', 'logo', 'change_1d', 'change_7d', 'change_1m', 'change_7dover7d', 'change_30dover30d', 'total24h', 'total48hto24h', 'total7d', 'total30d', 'total14dto7d', 'total60dto30d', 'total1y', 'average1y', 'totalAllTime', 'breakdown24h', 'chains', 'protocolType', 'methodologyURL', 'methodology', 'parentProtocol', 'latestFetchIsOk', 'versionKey', 'dailyRevenue', 'dailyUserFees', 'dailyHoldersRevenue', 'dailyCreatorRevenue', 'dailySupplySideRevenue', 'dailyProtocolRevenue', 'dailyBribesRevenue', 'dailyTokenTaxes', 'dailyFees', 'holdersRevenue30d', 'totalVolume7d', 'totalVolume30d'])

In [130]:
df = pd.DataFrame(data)

In [135]:
drop = ['methodology', 'breakdown24h', 'chains']
df = df.drop( columns = drop )

In [139]:
df = df.sort_values(by='total24h', ascending = False )

In [134]:
df.iloc[0]

defillamaId                                                            2685
name                                                   Arbitrum Exchange V2
disabled                                                              False
displayName                                            Arbitrum Exchange V2
module                                                     ArbitrumExchange
category                                                              Dexes
logo                      https://icons.llamao.fi/icons/protocols/arbitr...
change_1d                                                            -19.04
change_7d                                                            131.19
change_1m                                                            189.77
change_7dover7d                                                       40.26
change_30dover30d                                                     26.39
total24h                                                          20.586549
total48hto24

In [140]:
with connect_db() as connection:
    df.to_sql('general', con = connection, if_exists = 'replace' )